In [1]:
import numpy as np
import math
import tensorflow as tf


In [2]:
def uSoln(u0, ds, inFilt, pFilt, b):
    inChan, row, col = u0.shape
    tMax, outChan = inFilt.shape[0], inFilt.shape[2]
    u = np.zeros((tMax, outChan, row, col))
    x = np.zeros((tMax, outChan, row, col))
    # y=np.zeros((tMax,outChan,row,col))
    bzero = np.zeros((outChan))
    # print('outChan=',outChan,', u shape:',u.shape)

    # print('inFilt shape=',inFilt.shape,', u0 shape:',u0.shape)
    u[0] = conv(u0, inFilt[0], bzero)
    # print('u0',u[0])
    for h in range(tMax - 1):
        #print('h=',h)
        x[h] = act(u[h])
        zInt = np.array([conv(x[s], pFilt[s, h], b[h] / ((h + 1) * ds)) for s in range(h + 1)])
        if(h==2):
            print(zInt.shape)
        z = trapInt(zInt, ds)
        #print("z[with h^]: \n", z)
        u[h + 1] = z + conv(u0, inFilt[h + 1], bzero)
    x[tMax - 1] = act(u[tMax - 1])
    return x, u

In [3]:
def im2col(image, kerRow, kerCol, st):
    """
    Returns:
      col: (new_h*new_w,kerRow*KerCol*imageChan) matrix,
            each column is a cube that will convolve with a filter
            new_h=(imageRow-kerRow)//st+1, new_w=(imagrCol-kerCol)//st+1
    """

    chan, row, col = image.shape
    new_h = (row - kerRow) // st + 1
    new_w = (col - kerCol) // st + 1
    col = np.zeros([new_h * new_w, chan * kerRow * kerCol])

    for i in range(new_h):
        for j in range(new_w):
            patch = image[..., i * st:i * st + kerRow, j * st:j * st + kerCol]
            col[i * new_w + j, :] = np.reshape(patch, -1)
    return col

In [4]:
def mul(num):
    p = 1
    for m in num:
        p *= m
    # print('p=',p)
    return p

In [5]:
def col2im(ma, h_prime, w_prime, C):
    """
      Args:
      ma: (h_prime*w_prime*w,F) matrix, each col should be reshaped to C*h_prime*w_prime when C>0, or h_prime*w_prime when C = 0
      h_prime: reshaped filter height
      w_prime: reshaped filter width
      C: reshaped filter channel, if 0, reshape the filter to 2D, Otherwise reshape it to 3D
    Returns:
      if C == 0: (F,h_prime,w_prime) matrix
      Otherwise: (F,C,h_prime,w_prime) matrix
    """
    F = ma.shape[1]
    if (C == 1):
        out = np.zeros([F, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (h_prime, w_prime))
    else:
        out = np.zeros([F, C, h_prime, w_prime])
        for i in range(F):
            col = ma[:, i]
            out[i, :, :] = np.reshape(col, (C, h_prime, w_prime))

    return out

In [6]:
def conv(image, filt, b, st=1):
    imChan, imRow, imCol = image.shape
    inChan, outChan, kerRow, kerCol = filt.shape
    r = (imRow - 1) // st + 1
    c = (imCol - 1) // st + 1
    out = np.zeros((outChan, r, c))

    imPad = np.pad(image, ((0, 0), ((kerRow - 1) // 2, (kerRow - 1) // 2),
                           ((kerCol - 1) // 2, (kerCol - 1) // 2)))
    imMat = im2col(imPad, kerRow, kerCol, st)
    filtCol = np.reshape(filt, (outChan, -1))
    outCol = imMat.dot(filtCol.T) + b
    out = col2im(outCol, r, c, 1)
    return out

In [7]:
def act(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1] * layer[c, i, j]
                else:
                    out[c, i, j] = actWt[0] * layer[c, i, j]
    return out

In [8]:
def actInv(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = layer[c, i, j] / actWt[1]
                else:
                    out[c, i, j] = layer[c, i, j] / actWt[0]
    return out

In [9]:
def actD(layer):
    chan, row, col = layer.shape
    out = np.zeros((chan, row, col))

    for c in range(chan):
        for i in range(row):
            for j in range(col):
                if layer[c, i, j] >= 0:
                    out[c, i, j] = actWt[1]
                else:
                    out[c, i, j] = actWt[0]
    return out

In [10]:
def trapInt(vals, ds):
    return ds / 2 * (np.sum(vals, axis=0) + np.sum(vals[1:-1], axis=0))
    #trapezoidal rule: (1/2)*ds*(f(x_0) + 2f(x_1) + 2f(x_2) + ... + 2f(x_n-1) + f(x_n))

In [11]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [12]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    bzero = tf.zeros([inFilt.shape[4]])
    u0 = TFconv(raw_u0, inFilt[0], bzero)
    #for loop begins, h=0
    x0 = tf.nn.leaky_relu(u0, alpha=0.1)
    zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
    z0 = trapezoidIntegral(zInt0, ds)
    u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
    #h=1
    x1 = tf.nn.leaky_relu(u1, alpha=0.1)
    zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
    zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
    #zInt1 = zInt1_0 + zInt1_1
    zInt1 = tf.stack([zInt1_0, zInt1_1])
    z1 = trapezoidIntegral(zInt1, ds)
    u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
    #h=2
    x2 = tf.nn.leaky_relu(u2, alpha=0.1)#
    zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
    zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
    zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
    #zInt2 = zInt2_0 + zInt2_1 + zInt2_2
    zInt2 = tf.stack([zInt2_0, zInt2_1, zInt2_2])
    print(zInt2.shape)
    z2 = trapezoidIntegral(zInt2, ds)#
    u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
    #h=3
    x3 = tf.nn.leaky_relu(u3, alpha=0.1)
    zInt3_0 = TFconv(x0, pFilt[0,3], b[3] / ((3+1)*ds)) #s=0
    zInt3_1 = TFconv(x1, pFilt[1,3], b[3] / ((3+1)*ds)) #s=1
    zInt3_2 = TFconv(x2, pFilt[2,3], b[3] / ((3+1)*ds)) #s=2
    zInt3_3 = TFconv(x3, pFilt[3,3], b[3] / ((3+1)*ds)) #s=3
    #zInt3 = zInt3_0 + zInt3_1 + zInt3_2 + zInt3_3
    zInt3 = tf.stack([zInt3_0, zInt3_1, zInt3_2, zInt3_3])
    z3 = trapezoidIntegral(zInt3, ds)
    u4 = z3 + TFconv(raw_u0, inFilt[4], bzero)
    #end loop
    x4 = tf.nn.leaky_relu(u4, alpha=0.1)
    
    #print("z with h=0", z0)
    #print("z with h=1", z1)
    #print("z with h=2", z2)
    #print("z with h=3", z3)
    
    x = tf.stack([x0,x1,x2,x3,x4])
    u = tf.stack([u0,u1,u2,u3,u4])
    
    return x,u


In [35]:
t0 = 4
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



#np.random.seed(42)
np.set_printoptions(suppress=True)
actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

rng = np.random.default_rng(42)
uIn = rng.standard_normal(input_shape, dtype=np.float32)
iFilt = rng.standard_normal((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.standard_normal((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.standard_normal((t1,1), dtype=np.float32)

xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
print(uOut)

(3, 1, 5, 5)
[[[[ 0.95161361  2.43567809 -0.3856678  -1.67032042  0.00031825]
   [ 2.88517609  2.13933586 -3.96420572 -1.12482347  0.659127  ]
   [ 0.53239999 -2.69018429  0.19213539  1.94674919  0.17811383]
   [-3.62031171  1.72176612  4.82760097  2.25688488 -2.21752379]
   [ 0.72034023  0.01188322  0.37456762 -2.45935707 -0.16356799]]]


 [[[-2.93077919 -1.22818741 -3.41758207 -1.84771176  2.026555  ]
   [-2.90811882 -1.61863309  2.36663055  5.0050611  -0.46067193]
   [-0.91490982 -3.28435032  5.97902231  2.63123962 -0.51302317]
   [ 2.14895417  4.83561227 -8.58266892 -4.3518668  -1.84026138]
   [-0.54865177 -1.84955171  0.82515621  2.37375573  5.46253739]]]


 [[[ 1.19202826  6.21102218  3.13375793 -0.84376418 -1.52329009]
   [-7.17274501 -5.93612679  0.54477299  2.77155271  1.6417269 ]
   [-1.33779074 -4.40980764 -2.73762743  4.1358561   2.16300636]
   [ 1.02915427  2.04671009  2.04050864  1.28222798 -0.43683755]
   [-2.74673603 -0.38681147 -2.99540356 -1.87139023 -1.32362666]]]




In [36]:
#TFuIn = tf.transpose()
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)
TFiFilt = tf.transpose(tf.convert_to_tensor(iFilt, dtype=tf.float32), perm=[0,3,4,1,2])
TFpFilt = tf.transpose(tf.convert_to_tensor(paramFilt, dtype=tf.float32), perm=[0,1,4,5,2,3])
TFbiasVec = tf.convert_to_tensor(biasVec, dtype=tf.float32)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(TFuOut, perm=[4,0,1,2,3]))

(3, 1, 5, 5, 1)
tf.Tensor(
[[[[[ 0.9516136   2.435678   -0.3856678  -1.6703204   0.00031823]
    [ 2.8851762   2.1393359  -3.9642057  -1.1248233   0.65912706]
    [ 0.5324     -2.6901844   0.19213542  1.9467492   0.17811385]
    [-3.6203117   1.7217662   4.8276005   2.2568848  -2.2175238 ]
    [ 0.72034025  0.01188321  0.37456763 -2.4593573  -0.16356802]]]


  [[[-2.9307792  -1.2281874  -3.417582   -1.8477119   2.026555  ]
    [-2.9081187  -1.6186329   2.366631    5.005061   -0.46067193]
    [-0.9149097  -3.2843504   5.979022    2.6312397  -0.51302314]
    [ 2.1489542   4.835612   -8.582668   -4.3518667  -1.8402613 ]
    [-0.5486518  -1.8495516   0.8251563   2.3737555   5.4625373 ]]]


  [[[ 1.192028    6.2110214   3.1337578  -0.8437642  -1.5232902 ]
    [-7.1727448  -5.9361267   0.54477304  2.7715528   1.641727  ]
    [-1.3377907  -4.409808   -2.7376275   4.135856    2.1630063 ]
    [ 1.0291543   2.04671     2.0405087   1.282228   -0.43683746]
    [-2.746736   -0.38681155 -2.9954038  